<a href="https://colab.research.google.com/github/rmcnew/Pneumonia_Classifier/blob/master/Pneumonia_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 list 

In [0]:
# Richard Scott McNew
# A02077329
# CS 6600: Intelligent Systems

from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import drive
import os
import pathlib
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# enable accelerated linear algebra
tf.config.optimizer.set_jit(True)
# enable tensorflow AUTOTUNE
AUTOTUNE = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
IMAGE_SIDE = 450 
SHUFFLE_SIZE = 25
EPOCHS = 2
MODEL_PATH = "/content/drive/My Drive/USU/intelligent_systems/Pneumonia_Classifier/pneumonia_classifier_model"
DATASET_PATH = "/content/drive/My Drive/USU/intelligent_systems/Pneumonia_Classifier/dataset"

# mount drive
drive.mount('/content/drive')

# dataset paths
dataset = pathlib.Path(DATASET_PATH)
test = dataset.joinpath("test")
test_count = len(list(test.glob('**/*.jpeg')))
train = dataset.joinpath("train")
train_count = len(list(train.glob('**/*.jpeg')))
validate = dataset.joinpath("validate")
validate_count = len(list(validate.glob('**/*.jpeg')))


def create_train_image_generator():
    train_image_generator = ImageDataGenerator(rescale=1./255, zoom_range=0.5)
    train_data_gen = train_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(train), 
            shuffle=True, 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return train_data_gen

def create_test_image_generator():
    test_image_generator = ImageDataGenerator(rescale=1./255)
    test_data_gen = test_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(test), 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return test_data_gen

def create_validate_image_generator():
    validate_image_generator = ImageDataGenerator(rescale=1./255)
    validate_data_gen = validate_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(validate), 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return validate_data_gen

# val 90
#def create_model():
#    model = Sequential([
#        Conv2D(450, 10, padding='same', activation='relu', kernel_regularizer='l2', input_shape=(450, 450, 3)),
#        MaxPooling2D(),
#        Conv2D(225, 5, padding='same', activation='relu', kernel_regularizer='l2'),
#        MaxPooling2D(),
#        Dropout(0.2),
#        Conv2D(100, 5, padding='same', activation='relu', kernel_regularizer='l2'),
#        MaxPooling2D(),        
#        Flatten(),
#        Dense(512, activation='relu'),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy']) 
#    return model


def create_model():
    model = Sequential([
        Conv2D(IMAGE_SIDE, 10, padding='same', activation='relu', kernel_regularizer='l2', input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3)),
        MaxPooling2D(),
        Conv2D(225, 5, padding='same', activation='relu', kernel_regularizer='l2'),
        MaxPooling2D(),
        Dropout(0.2),
        Conv2D(100, 5, padding='same', activation='relu', kernel_regularizer='l2'),
        MaxPooling2D(),        
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy']) 
    return model

def train_model():
    train_data_gen = create_train_image_generator()
    test_data_gen = create_test_image_generator()
    model = create_model()
    model.fit(
        train_data_gen,        
        steps_per_epoch=train_count // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_data_gen,        
        validation_steps=test_count // BATCH_SIZE
    ) 
    model.save(MODEL_PATH)
    

def unmount_drive():
  drive.flush_and_unmount()

def load_model(path):
    model = tf.keras.models.load_model(path, custom_objects=None, compile=True)
    return model

def load_trained_model():
    model = tf.keras.models.load_model(MODEL_PATH, custom_objects=None, compile=True)
    return model

def train_model_more():
    train_data_gen = create_train_image_generator()
    test_data_gen = create_test_image_generator()
    model = load_trained_model()
    model.fit(
        train_data_gen,
        steps_per_epoch=train_count // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_data_gen,
        validation_steps=test_count // BATCH_SIZE
    )
    model.save(MODEL_PATH)
    

train_model_more()
unmount_drive()